# Model Creator Example

Demo of how to build a CIM model from scratch

## Import Libraries and Connect to Database

In [ ]:
# Specify the CIM profile version, using GridAPPS-D extensions, ver. 2021
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)
sparql = importlib.import_module('cimgraph.loaders.sparql.' + cim_profile)

In [ ]:
# Import Distributed Topology Processor
from topology_processor import DistributedTopology

In [ ]:
# Import CIMantic Graphs database connection and models
from cimgraph.loaders import Parameter, ConnectionParameters
from cimgraph.loaders.blazegraph.blazegraph import BlazegraphConnection
from cimgraph.models import NewModel, DistributedModel
import json
import uuid


In [ ]:
# Connect to Blazegraph Database
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params, 'rc4_2021')

## Create a New Distribution Network

### Create Feeder Object

In [ ]:
# Create new network object and connect to database for upload
fdr_mrid = str(uuid.uuid4()) # auto-generate unique id
fdr = cim.Feeder(name = 'demo_feeder', mRID = fdr_mrid) # create feeder

network = NewModel(bg) # Create CIM-Graph model
network.Feeder = fdr
network.add_to_typed_catalog([fdr])

In [ ]:
# Print feeder python object
print(fdr)

In [ ]:
# Dump all attributed of feeder object including empty fields
json.loads(network.__dumps__(cim.Feeder))

In [ ]:
# Pretty-print feeder object with empty fields hidden
network.pprint(cim.Feeder)

### Specify Substation and Geographic Region

In [ ]:
# Create Substation and GeographicalRegion objects
reg = cim.GeographicalRegion(name = 'demo_region', mRID = str(uuid.uuid4()))
subreg = cim.SubGeographicalRegion(name = 'demo_subregion', mRID = str(uuid.uuid4()), Region=reg)
sub = cim.Substation(name = 'demo_sub', mRID = str(uuid.uuid4()), Region=subreg)

In [ ]:
# Add to network model
network.add_to_typed_catalog([reg, subreg, sub])
# Associate substation to feeder
fdr.NormalEnergizingSubstation = sub
# Print full feeder object
print(fdr)

In [ ]:
# Create reverse association
sub.NormalEnergizedFeeder = fdr

In [ ]:
# Printing full object results in infinite print loop
print(fdr)

In [ ]:
# Pretty-print feeder object with empty fields hidden
network.pprint(cim.Feeder)
network.pprint(cim.Substation)

### Create a new line

In [ ]:
# Create a new line
line = cim.ACLineSegment(name = "new_line", mRID = str(uuid.uuid4()))
line.EquipmentContainer = fdr # Associate with feeder
line.r = 0.004
line.x = 0.080
line.bch = 0.006

In [ ]:
# Add to CIM-Graph network model
network.add_to_typed_catalog([line])
network.pprint(cim.ACLineSegment)

### Create a line builder script
Semi-automated script to create ACLineSegment and associated buses

In [ ]:
def create_line(network, name:str, bus1:str, bus2:str):
    line = cim.ACLineSegment(name = name, mRID = str(uuid.uuid4()), EquipmentContainer=network.Feeder)
    node1 = cim.ConnectivityNode(name = bus1, mRID = str(uuid.uuid4()), ConnectivityNodeContainer=network.Feeder)
    node2 = cim.ConnectivityNode(name = bus2, mRID = str(uuid.uuid4()), ConnectivityNodeContainer=network.Feeder)
    term1 = cim.Terminal(mRID = str(uuid.uuid4()))
    term2 = cim.Terminal(mRID = str(uuid.uuid4()))

    term1.ConnectivityNode = node1
    term2.ConnectivityNode = node2
    term1.ConductingEquipment = line
    term2.ConductingEquipment = line
    node1.Terminals.append(term1)
    node2.Terminals.append(term2)
    line.Terminals.append(term1)
    line.Terminals.append(term2)

    network.add_to_typed_catalog([line, node1, node2, term1, term2])

In [ ]:
create_line(network, 'line_1_2', 'bus_1', 'bus_2')
create_line(network, 'line_3_4', 'bus_3', 'bus_4')

In [ ]:
network.pprint(cim.ACLineSegment)

### Batch Edit all Line Impedances

In [ ]:
# Double impedance of all lines
for line in network.typed_catalog[cim.ACLineSegment].values():
    line.r = 0.004
    line.x = 0.080
    line.bch = 0.006

In [ ]:
network.pprint(cim.ACLineSegment)

## Traverse the New Model

In [ ]:
# See the two buses connected to the line
print(line.Terminals[0].ConnectivityNode.name)
print(line.Terminals[1].ConnectivityNode.name)

In [ ]:
# See the substation feeding the line
print(line.EquipmentContainer.name)
print(line.EquipmentContainer.NormalEnergizingSubstation.name)

## Export and Upload

In [ ]:
# Write to XML File
dest_file = 'demo_feeder.xml'
schema = 'http://iec.ch/TC57/CIM100#'
network.write_xml(dest_file, schema)

In [ ]:
# Upload to Blazegraph Database
network.upload()

In [ ]:
# Use CIMHub Library to print all feeders in database
!python3 CIMHub/src_python/cimhub/ListFeeders.py